In [1]:
import os
import fnmatch
from sklearn.model_selection import train_test_split

data_dir = 'Bilder'
file_list = os.listdir(data_dir)
image_paths = []
steering_angles = []
pattern = "*.jpg"
for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        image_paths.append(os.path.join(data_dir,filename))
        angle = int(filename[-7:-4])  # AAA part of timestamp_AAA.jpg --> AAA is the angle in range 045 - 135
        steering_angles.append(angle)

X_train, X_valid, y_train, y_valid = train_test_split( image_paths, steering_angles, test_size=0.2)
print(f"Training data: {len(X_train)} Validation data: {len(X_valid)}")

Training data: 270 Validation data: 68


In [ ]:
! pip3 install -U numpy